In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import requests
import xml.etree.ElementTree as ET
import time
import datetime
import os.path

from os import path
from apscheduler.schedulers.blocking import BlockingScheduler
from apscheduler.triggers.cron import CronTrigger

In [2]:
FILE_PATH='request_juvent/'

In [3]:
def add_juvent_data():
    file_name=FILE_PATH+'Juvent_'+datetime.datetime.now().strftime("%b")+'_'+str(datetime.datetime.now().year)+'.csv'
    
    if  path.exists(file_name):
        
        try:
            df=pd.read_csv(file_name).drop('Unnamed: 0',axis=1)
        except:
            df=pd.read_csv(file_name)
    else:
        cols=['Time', 'Turbine_id','Production', 'Power', 'Wind_Speed','Wind_Direction', 'Temperature']
        df=pd.DataFrame(columns=cols)

    response=requests.get('http://www.juvent.ch/tl_files/content/atfront_flash/normalizedTurbine.xml')
    tree = ET.fromstring(response.content)
    for child in tree:
        content=[datetime.datetime.now().__str__()]
        content.append(child.attrib['id'])
        for i in child.itertext():
            if i.strip()=='':
                pass
            else:
                content.append(i.strip())

        df=df.append(pd.Series(content, index=df.columns), ignore_index=True)
    df.to_csv(file_name)
    del df

In [ ]:
sched = BlockingScheduler()
sched.add_job(add_juvent_data,CronTrigger.from_crontab('*/10 * * * *'))
sched.start()

Job "add_juvent_data (trigger: cron[month='*', day='*', day_of_week='*', hour='*', minute='*/10'], next run at: 2020-08-04 17:40:00 CEST)" raised an exception
Traceback (most recent call last):
  File "C:\Users\students\Anaconda3\envs\wnn\lib\site-packages\urllib3\connection.py", line 157, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw
  File "C:\Users\students\Anaconda3\envs\wnn\lib\site-packages\urllib3\util\connection.py", line 84, in create_connection
    raise err
  File "C:\Users\students\Anaconda3\envs\wnn\lib\site-packages\urllib3\util\connection.py", line 74, in create_connection
    sock.connect(sa)
TimeoutError: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\students\Anaconda3\envs